In [2]:
%%capture
import networkx as nx
import pandas as pd
import spotipy
from spotipy . oauth2 import SpotifyClientCredentials
import matplotlib.pyplot as plt
from tqdm import tqdm
import time
import random
from spotipy.exceptions import SpotifyException
import scipy

In [3]:
CLIENT_ID = "c7f567959a73413b9ddf798895e1fe39"
CLIENT_SECRET = "e0c3fef273a7409793a1760d7bf9e350"

auth_manager = SpotifyClientCredentials (client_id = CLIENT_ID, client_secret = CLIENT_SECRET)
sp = spotipy . Spotify ( auth_manager = auth_manager )

## Define the problem

The goal is to discover the most influential artists within different music genres, for a range of genres defined by Spotify. The influential artists would be those who have high popularity and follower counts, and a large number of related artists.


## Data Acquisition

In [20]:
genre_list = sp.recommendation_genre_seeds()
counter = 0
for i in genre_list.values():
    for j in i:
        counter += 1
        print(j)
        

acoustic
afrobeat
alt-rock
alternative
ambient
anime
black-metal
bluegrass
blues
bossanova
brazil
breakbeat
british
cantopop
chicago-house
children
chill
classical
club
comedy
country
dance
dancehall
death-metal
deep-house
detroit-techno
disco
disney
drum-and-bass
dub
dubstep
edm
electro
electronic
emo
folk
forro
french
funk
garage
german
gospel
goth
grindcore
groove
grunge
guitar
happy
hard-rock
hardcore
hardstyle
heavy-metal
hip-hop
holidays
honky-tonk
house
idm
indian
indie
indie-pop
industrial
iranian
j-dance
j-idol
j-pop
j-rock
jazz
k-pop
kids
latin
latino
malay
mandopop
metal
metal-misc
metalcore
minimal-techno
movies
mpb
new-age
new-release
opera
pagode
party
philippines-opm
piano
pop
pop-film
post-dubstep
power-pop
progressive-house
psych-rock
punk
punk-rock
r-n-b
rainy-day
reggae
reggaeton
road-trip
rock
rock-n-roll
rockabilly
romance
sad
salsa
samba
sertanejo
show-tunes
singer-songwriter
ska
sleep
songwriter
soul
soundtracks
spanish
study
summer
swedish
synth-pop
tango
techno

In [30]:
counter = 0

G = nx.DiGraph()

for i in genre_list.values():
    for j in i:
        counter += 1
        if counter <= 5:
            top_artists = sp.recommendations(seed_genres=[j])['tracks']
            artists = [track['artists'][0] for track in top_artists]
        else:
            break
        
        for artist in artists:
            artist_data = sp.artist(artist['id'])
            followers = artist_data['followers']['total']
            popularity = artist_data['popularity']
            genres = artist_data['genres']  # Get the genres of the artist
            related_artists = sp.artist_related_artists(artist['id'])['artists']

            # Add a node for the artist into the graph and store its attributes
            G.add_node(artist['id'], name=artist_data['name'], followers=followers, popularity=popularity, genres=genres)

            # For each related artist, add an edge from the artist to the related artist
            for related_artist in related_artists:
                G.add_edge(artist['id'], related_artist['id'])


In [32]:
# Convert the genres list to a string for each artist
for node, data in G.nodes(data=True):
    if 'genres' in data:
        data['genres'] = ', '.join(data['genres'])
        
# Then you can write the graph to a GraphML file
nx.write_graphml(G, "artists_graph.graphml")

In [44]:
# Create an empty DataFrame
import pandas as pd
import networkx as nx

df = pd.DataFrame(columns=['id', 'name', 'followers', 'popularity', 'genres'])

# Add each artist to the DataFrame
for node, data in G.nodes(data=True):
    if not data:  # if data dictionary is empty, skip this iteration
        continue
    new_row = pd.DataFrame([[node, data['name'], data['followers'], data['popularity'], data['genres']]], columns=['id', 'name', 'followers', 'popularity', 'genres'])
    df = pd.concat([df, new_row], ignore_index=True)

# Write the data to a CSV file
df.to_csv('artists_data.csv', index=False)



## Data Preprocessing

## Data Analysis

## Visualization

## Discussion of results